In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import random
import seaborn as sns

import sys
import os
sys.path.append('../')

from VQUnet3D import Net as QNet
from Unet3D import Net as Net
from skimage.util import random_noise


MONAI version: 0.8.0
Numpy version: 1.21.5
Pytorch version: 1.10.2+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.6
Nibabel version: 3.2.2
scikit-image version: 0.19.1
Pillow version: 8.4.0
Tensorboard version: 2.4.0
gdown version: 4.2.1
TorchVision version: 0.8.2
tqdm version: 4.62.3
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.3.5
einops version: 0.4.0
transformers version: 4.16.2
mlflow version: 1.23.1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies

/tmp/tmpgqjvtmvc
MONAI version: 0.8.0
Numpy version: 1.21.5
Pytorch version: 1.10.2+cu102
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: 0.4.6
Nibabel version: 3.2.2
scikit-image version

In [2]:
net = Net(fold=0)
net.load_state_dict(torch.load('/vol/biomedic2/agk21/PhDLogs/codes/Vector-Quantisation-for-Robust-Segmentation/Logs/output4/disease1/vqseg1/version_1/checkpoints/epoch=289-step=6959.ckpt')['state_dict'])
net = net.eval()
net = net.cuda()

In [3]:
from Unet3D import Data
loader = Data(fold=0,numfolds=5, size = (32, 32, 32))
val_dataloader = loader.val_dataloader()


24


Loading dataset: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:17<00:00,  2.92s/it]


In [4]:
def noisy(noise_typ, image, weightage = 0.0):
    if noise_typ == "gauss":
        mean = 0
        var = 1
        sigma = var**0.5
        gauss = np.random.normal(mean, sigma,image.shape)
        gauss = gauss.reshape(*image.shape)
        noisy = image + weightage*gauss
        return noisy
    elif noise_typ == "s&p":
        s_vs_p = 0.5
        amount = weightage
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape[-3:]]
        for x, y, z in zip(*coords):
            out[0, 0, x, y, z] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape[-3:]]
        for x, y, z in zip(*coords):
            out[0, 0, x, y, z] = 0
            
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(weightage * image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        gauss = np.random.randn(*image.shape)
        gauss = gauss.reshape(*image.shape)        
        noisy = image + weightage*image * gauss
        return noisy
    

In [5]:
import pickle


def generate_TTA_results(name, data, noise_type='gauss', base_path='.'):
    nTTAs = 100
    zdata = {}
    
    assert noise_type in ['gauss', 's&p', 'poisson', 'speckle'], 'invalid noise_type'
    
    noise_threshold_list = [0.0, 0.01, 0.1, 0.5, 0.75]
    
    for noise_threshold in noise_threshold_list:
        embs = []; recons = []
        for _ in range(nTTAs):
            x = data['image'][:1, ...].numpy()
            dtype = data['image'].dtype
            x = noisy(noise_type, x, noise_threshold)
            x = torch.tensor(x).type(dtype).cuda() 
            with torch.no_grad():
                x, encoding = net._model(x)
            embs.append(encoding)
            recons.append(x)

        embs = torch.cat(embs, 0)
        recons = torch.cat(recons, 0)
        recons = torch.argmax(recons, 1)
        
        zdata[noise_threshold] = {'emb': embs.cpu().numpy(), 
                                      'recon': recons.cpu().squeeze().numpy(), 
                                      'img': data['image'][:1, ...].squeeze().numpy(), 
                                      'label': data['label'][:1, ...].squeeze().numpy()}

    base_path = os.path.join(base_path, noise_type)
    os.makedirs(base_path, exist_ok=True)
    with open(os.path.join(base_path,'{}.pickle'.format(name)), 'wb') as file:
        pickle.dump(zdata, file)
    return zdata

In [ ]:
from tqdm import tqdm

all_zdata = {}
for i in tqdm(range(100)):
    data = next(iter(val_dataloader))
    all_zdata[i] = {}
    all_zdata[i]['gauss'] = generate_TTA_results('image{}'.format(i), data, 'gauss', 'ABDCT/UNet')
    all_zdata[i]['s&p'] = generate_TTA_results('image{}'.format(i), data, 's&p', 'ABDCT/UNet')
    all_zdata[i]['poisson'] = generate_TTA_results('image{}'.format(i), data, 'poisson', 'ABDCT/UNet')
    all_zdata[i]['speckle'] = generate_TTA_results('image{}'.format(i), data, 'speckle', 'ABDCT/UNet')
    
with open('all_data_ABDCT_UNet.pickle', 'wb') as file:
    pickle.dump(all_zdata, file)

 28%|███████████████████████████████████████████▉                                                                                                                 | 28/100 [54:28<2:34:13, 128.52s/it]

In [ ]:
!nvidia-smi

In [ ]:

qnet = QNet()
qnet.load_state_dict(torch.load('/vol/biomedic2/agk21/PhDLogs/codes/Vector-Quantisation-for-Robust-Segmentation/Logs/output4/disease1/vqseg/version_1/checkpoints/epoch=294-step=7079.ckpt')['state_dict'])
qnet = qnet.eval()

In [ ]:
import pickle

def VQgenerate_TTA_results(name, data, noise_type='gauss', base_path='.'):
    nTTAs = 100
    zdata = {}
    
    assert noise_type in ['gauss', 's&p', 'poisson', 'speckle'], 'invalid noise_type'
    
    noise_threshold_list = [0.0, 0.01, 0.1, 0.5, 0.75]
    
    for noise_threshold in noise_threshold_list:
        embs = []; recons = []; emb_losses = []
        for _ in range(nTTAs):
            x = data['image'][:1, ...].numpy()
            dtype = data['image'].dtype
            x = noisy(noise_type, x, noise_threshold)
            x = torch.tensor(x).type(dtype).cuda() 
            with torch.no_grad():
                x, emb_loss, encoding = qnet._model(x)
            embs.append(encoding)
            recons.append(x)
            emb_losses.append(emb_loss)

        embs = torch.cat(embs, 0)
        emb_losses = torch.cat(emb_losses, 0)
        recons = torch.cat(recons, 0)
        recons = torch.argmax(recons, 1)
        
        zdata[noise_threshold] = {'emb': embs.cpu().numpy(),
                                      'emb_loss': emb_losses.cpu().numpy(),
                                      'recon': recons.cpu().squeeze().numpy(), 
                                      'img': data['image'][:1, ...].squeeze().numpy(), 
                                      'label': data['label'][:1, ...].squeeze().numpy()}

    base_path = os.path.join(base_path, noise_type)
    os.makedirs(base_path, exist_ok=True)
    with open(os.path.join(base_path,'{}.pickle'.format(name)), 'wb') as file:
        pickle.dump(zdata, file)
    return zdata

In [ ]:
from tqdm import tqdm

all_zqdata = {}
for i in tqdm(range(100)):
    data = next(iter(val_dataloader))
    all_zqdata[i] = {}
    all_zqdata[i]['gauss'] = VQgenerate_TTA_results('image{}'.format(i), data, 'gauss', 'ABDCT/VQNet')
    all_zqdata[i]['s&p'] = VQgenerate_TTA_results('image{}'.format(i), data, 's&p', 'ABDCT/VQNet')
    all_zqdata[i]['poisson'] = VQgenerate_TTA_results('image{}'.format(i), data, 'poisson', 'ABDCT/VQNet')
    all_zqdata[i]['speckle'] = VQgenerate_TTA_results('image{}'.format(i), data, 'speckle', 'ABDCT/VQNet')
    
with open('all_data_ABDCT_VQNet.pickle', 'wb') as file:
        pickle.dump(all_zqdata, file)

In [ ]:
zqdata[0.01]['emb'].shape, zdata[0.01]['emb'].shape

In [ ]:
for noise_threshold in noise_threshold_list:
    print ("============================ noise: {} ===========".format(noise_threshold))
    z, zq = zdata[noise_threshold]['emb'], zqdata[noise_threshold]['emb']
    
    plt.figure(figsize=(50, 50))
    plt.subplot(1, 2, 1)
    plt.imshow(z[:, :, 0,0,0], cmap='coolwarm')
    
    
    plt.subplot(1, 2, 2)
    plt.imshow(zq[:, :, 0,0,0], cmap='coolwarm')
    plt.show()
    
    
    plt.figure(figsize=(50, 50))
    plt.subplot(1, 2, 1)
    plt.imshow(z[:, :, 1, 1, 1], cmap='coolwarm')
    
    
    plt.subplot(1, 2, 2)
    plt.imshow(zq[:, :, 1, 1, 1], cmap='coolwarm')
    plt.show()
    
    plt.figure(figsize=(50, 50))
    plt.subplot(1, 2, 1)
    plt.imshow(z[:, :, 1, 0, 1], cmap='coolwarm')
    
    plt.subplot(1, 2, 2)
    plt.imshow(zq[:, :, 1, 0, 1], cmap='coolwarm')
    plt.show()
    
    plt.figure(figsize=(50, 50))
    plt.subplot(1, 2, 1)
    plt.imshow(z[:, :, 1, 0, 0], cmap='coolwarm')
    
    
    plt.subplot(1, 2, 2)
    plt.imshow(zq[:, :, 1, 0, 0], cmap='coolwarm')
    plt.show()


In [ ]:
mean_dice = lambda p, l: np.mean([2*(p == i)*(l == i)/((p==i) + (l ==i) + 1e-3) for i in np.unique(l)])
import pandas as pd

def _draw_(dict_info, noise_type='gauss', base_dir= '.'):
    nimgs = len(dict_info.keys())
    test_ = dict_info[0][noise_type]
    noise_threshold_list = list(test_.keys())
    
    test_embs = test_[noise_threshold_list[0]]['emb'] # nttas, ...
    nttas = test_embs.shape[0]
    z_dim = np.prod(test_embs.shape[1:])
    
    
    dice_scores_x = []
    dice_scores_y = []
    categories = []
    
    for ni, noise_threshold in enumerate(noise_threshold_list):
        
            
        result_img = np.zeros((nimgs, z_dim))
        
        for iimg in range(nimgs):
            z = dict_info[iimg][noise_type][noise_threshold]['emb'].reshape(nttas, -1)
            z = (z - z.min())/(z.max() - z.min())
            
            result_img[iimgs, :] = np.var(z, axis=0).T
            
            if noise_threshold == 0.0:
                dice_scores_x.append(np.mean([mean_dice(dict_info[iimg][noise_type][noise_threshold]['recon'][itta, ...], 
                                                    dict_info[iimg][noise_type][noise_threshold]['label']) \
                                                      for itta in range(nttas)]))
            else:
                dice_scores_y.append(np.mean([mean_dice(dict_info[iimg][noise_type][noise_threshold]['recon'][itta, ...], 
                                                    dict_info[iimg][noise_type][noise_threshold]['label']) \
                                                      for itta in range(nttas)]))
                
                categories.append('Noise Threshold: {}'.format(noise_threshold))
                
        
        base_path = os.path.join(base_path, noise_type)
        base_path = os.path.join(base_path, 'plots')
        os.makedirs(base_path, exist_ok=True)
        
        plt.figure(figsize=(1, 10))
        plt.imshow(result_img, cmap='coolwarm', vmin=0, vmax=1)
        plt.axis (‘off’)
        plt.tight_layout()
        plt.savefig(os.path.join(base_path, 'NoiseT{}_NoiseType{}.png'.format(noise_threshold, noise_type)))
        
        
    df = pd.DataFrame()
    df['actual-dice'] = dice_scores_x * (len(noise_threshold_list) - 2)
    df['perturbed- dice'] = dice_scores_y
    df['categories'] = categories

    p = sns.jointplot(data=df, x='actual-dice', y='perturbed- dice',  hue="categories", alpha=0.5)
    p.plot_joint(sns.kdeplot, levels=20)
    p.fig.suptitle("aligned latent vectors")
    p.fig.tight_layout()
    p.fig.subplots_adjust(top=0.95)
    plt.savefig(os.path.join(base_path, 'dice_score_distribution_{}.png'.format(noise_type)))

In [ ]:
with open('all_data_ABDCT_UNet.pickle', 'rb') as file:
    dict_info = pickle.load(file)
    
_draw_(dict_info, 'gauss', 'ABDCT/VQNet')
_draw_(dict_info, 's&p', 'ABDCT/VQNet')
_draw_(dict_info, 'poisson', 'ABDCT/VQNet')
_draw_(dict_info, 'speckle', 'ABDCT/VQNet')

In [ ]:
with open('all_data_ABDCT_VQNet.pickle', 'rb') as file:
    dict_info = pickle.load(file)
    
_draw_(dict_info, 'gauss', 'ABDCT/VQNet')
_draw_(dict_info, 's&p', 'ABDCT/VQNet')
_draw_(dict_info, 'poisson', 'ABDCT/VQNet')
_draw_(dict_info, 'speckle', 'ABDCT/VQNet')